In [1]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time 
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *
from analystcommunity.read_connection_data_warehouse import run_read_dwd_query

from datetime import datetime

from procurement_lib import GoogleSheet

In [2]:
paramss_csv=GoogleSheet("1cnuspMEt1MlnG7Io7Il8IiopXfJy0vmnalInhbJOQsk")
df = paramss_csv.get_as_dataframe('info_py')

In [3]:
# Convert to datetime
df['quotation_date'] = pd.to_datetime(df['quotation_date'])

In [4]:
def run_query(grupo,skus,fechai,fechaf):
    query = """
    WITH
    gmv AS (
    SELECT
        s.identifier_value AS region_code,
        SUM(fs.gmv_pxq) AS gmv
    FROM dpr_sales.fact_sales                   fs
    INNER JOIN dpr_shared.dim_site              s   ON s.site_id = fs.dim_site
    INNER JOIN dpr_shared.dim_product           dp  ON dp.product_id = fs.dim_product
    INNER JOIN dpr_shared.dim_stock_unit        su  ON su.product_id = fs.dim_product
    INNER JOIN dpr_shared.dim_stock_unit        sup  ON nvl(nullif(su.source_parent_id,0),su.source_id) = sup.source_id

    WHERE
        fs.gmv_enabled = TRUE
        AND fulfillment_order_status NOT IN ('CANCELLED', 'ARCHIVED','No value')
        AND fs.fb_order_status_id IN (1,6,7,8)
        AND fs.is_deleted = FALSE
        AND fs.dim_status = 1
        AND dp.is_slot = 'false'
        AND fs.gmv_pxq_local > 0
        AND DATE(fs.order_close_date) >= '{fecha_inicio}'
        AND DATE(fs.order_close_date) < '{fecha_fin}'   
    group by 1
    )

    SELECT
        s.identifier_value AS region_code,
        cat.parent_description AS cat,
        sup.source_id,
        sup.card_description AS product_name,
        '{group}' AS grupo,
        COUNT(DISTINCT fs.order_id) AS orders,
        COUNT(DISTINCT fs.dim_customer) AS customers,
        SUM(fs.product_quantity_x_step_unit) AS cant,
        SUM(fs.gmv_pxq_local)/4.75 AS gmv_usd,
        SUM(COALESCE(fsd.product_discount,0))/4.75 AS discount_applied,
        100.00*discount_applied/gmv_usd AS per_dct,
        (1-(SUM(COALESCE(cogs_p_mtd,inventory_p_fin)*fs.product_quantity_x_step_unit)/SUM(fs.product_price*fs.product_quantity_x_step_unit)))*100.00 AS margin,
        gmv_usd*margin/100.00 AS cash_margin,
        cash_margin-discount_applied AS net_cash_margin,
        100.00*net_cash_margin/gmv_usd AS net_margin,
        gmv_usd/AVG(gmv.gmv) as gmv_mix

    FROM dpr_sales.fact_sales                   fs
    INNER JOIN dpr_shared.dim_site              s   ON s.site_id = fs.dim_site
    INNER JOIN dpr_shared.dim_product           dp  ON dp.product_id = fs.dim_product
    INNER JOIN dpr_shared.dim_category          cat ON cat.category_id = dp.category_id
    LEFT JOIN dpr_sales.fact_sales_discounts    fsd ON fs.order_item_id = fsd.order_item_id 
    INNER JOIN dpr_shared.dim_stock_unit        su  ON su.product_id = fs.dim_product
    INNER JOIN dpr_shared.dim_stock_unit        sup  ON nvl(nullif(su.source_parent_id,0),su.source_id) = sup.source_id
    LEFT JOIN dpr_cross_business.fact_cross_business_insights m ON m.dim_stock_unit = sup.stock_unit_id AND m.dim_date = fs.dim_submitted_date
    INNER JOIN gmv ON s.identifier_value =  gmv.region_code
    --LEFT JOIN dpr_product_pricing.obt_benchmark_product_prices  pb  ON pb.stock_unit_id = sup.stock_unit_id AND DATE(fs.order_submitted_date) = pb.benchmark_date

    WHERE 
        fs.gmv_enabled = TRUE
        AND fulfillment_order_status NOT IN ('CANCELLED', 'ARCHIVED','No value')
        AND fs.fb_order_status_id IN (1,6,7,8)
        AND fs.is_deleted = FALSE
        AND fs.dim_status = 1
        AND dp.is_slot = 'false'
        AND fs.gmv_pxq_local > 0
        AND sup.source_id IN {skus}
        AND DATE(fs.order_close_date) >= '{fecha_inicio}'
        AND DATE(fs.order_close_date) < '{fecha_fin}'
    GROUP BY 1,2,3,4,5
    """.format(group=grupo,skus=tuple(skus),fecha_inicio=fechai,fecha_fin=fechaf)

    df1 = run_read_dwd_query(query)
    df1[['orders','customers', 'cant', 'gmv_usd', 'discount_applied', 'per_dct', 'margin',
       'cash_margin', 'net_cash_margin', 'net_margin', 'gmv_mix']] = df1[['orders','customers', 'cant', 'gmv_usd', 'discount_applied', 'per_dct', 'margin',
       'cash_margin', 'net_cash_margin', 'net_margin', 'gmv_mix']].astype('float32')
    return df1

In [5]:
unique_dates = list(df['quotation_date'].unique())

current_date = pd.to_datetime(datetime.now().strftime('%Y-%m-%d'))

unique_dates.append(current_date)
unique_dates.sort()

In [6]:
unique_dates

[numpy.datetime64('2024-06-21T00:00:00.000000000'),
 numpy.datetime64('2024-06-25T00:00:00.000000000'),
 numpy.datetime64('2024-06-28T00:00:00.000000000'),
 numpy.datetime64('2024-07-01T00:00:00.000000000'),
 Timestamp('2024-07-01 00:00:00')]

In [8]:
dfs = []
for i, fecha in enumerate(unique_dates[:-1]):  # iterate until the second last date
    if i == unique_dates[-1]:
        break
    df_aux = run_query('G'+str(i)+'_W-0', 
                       df.loc[df.quotation_date == fecha, 'source_id'].unique(), 
                       unique_dates[i], 
                       unique_dates[i+1])
    dfs.append(df_aux)

    # Iterate over the previous weeks
    for j in range(1, 5):  # Adjust the range as needed
        # Calculate start and end of the previous week
        start_of_week = unique_dates[i] - np.timedelta64(7 * j, 'D')
        end_of_week = unique_dates[i+1] - np.timedelta64(7 * j, 'D')

        df_aux = run_query('G'+str(i)+'_W-'+str(j), 
                       df.loc[df.quotation_date == fecha, 'source_id'].unique(), 
                       start_of_week, 
                       end_of_week)
        dfs.append(df_aux)

df_final = pd.concat(dfs, ignore_index=True)

In [9]:
df_sheet=GoogleSheet("1nfIXRK04dnuZM1jAMaZ9ZTWMIWzHWq5OIWQ9Ga9B5Vw")
df_sheet.set_with_dataframe(df_final, worksheet='data', autocreate=True)

In [10]:
# Define a function to apply the custom aggregation
def custom_ventas(group, column):
    return (100.00 * group[column] * group['gmv_mix']).sum()/group['gmv_mix'].sum()
merge_df2 = df_final.dropna(how='any')
# Group by 'city' and 'year_week', aggregate multiple columns with custom aggregation
df_final2 = merge_df2.groupby(['region_code','grupo']).apply(lambda group: pd.Series({
    'orders': np.round(group['orders'].mean(),0),
    'customers': np.round(group['customers'].mean(),0),
    'cant': group['cant'].sum(),
    'gmv_usd': group['gmv_usd'].sum(),
    'discount_applied': group['discount_applied'].sum(),
    'cash_margin': group['cash_margin'].sum(),
    'net_cash_margin': group['net_cash_margin'].sum(),
    'gmv_mix': group['gmv_mix'].sum(),  # Compute sum of gmv_mix directly
})).reset_index()


df_final2['per_dct'] = df_final2.discount_applied/df_final2.gmv_usd
df_final2['mg'] = df_final2.cash_margin/df_final2.gmv_usd
df_final2['nmg'] = df_final2.net_cash_margin/df_final2.gmv_usd
#df_final2 = df_final2.round(2)

In [11]:
df_sheet.set_with_dataframe(df_final2, worksheet='data2', autocreate=True)

In [12]:
df_final2

,region_code,grupo,orders,customers,cant,gmv_usd,discount_applied,cash_margin,net_cash_margin,gmv_mix,per_dct,mg,nmg
0,BHZ,G0_W-0,6.0,6.0,3203.000000,3186.282959,23.376701,536.343323,512.966614,0.0177,0.007337,0.168329,0.160992
1,BHZ,G0_W-1,9.0,9.0,4263.000000,4782.488770,23.122000,813.427124,790.305115,0.0261,0.004835,0.170084,0.165250
2,BHZ,G0_W-2,9.0,9.0,4513.000000,4979.387695,50.048203,793.817932,743.769714,0.0276,0.010051,0.159421,0.149370
3,BHZ,G0_W-3,7.0,7.0,3318.000000,3791.889160,15.831200,1033.403198,1017.571960,0.0228,0.004175,0.272530,0.268355
4,BHZ,G0_W-4,8.0,8.0,3818.000000,4045.520508,21.944801,688.428833,666.484070,0.0213,0.005424,0.170171,0.164746
5,BHZ,G1_W-0,7.0,7.0,2464.000000,2656.310303,2.835600,536.090576,533.255005,0.0107,0.001067,0.201818,0.200750
6,BHZ,G1_W-1,7.0,7.0,3034.000000,2941.983887,2.806300,568.990967,566.184692,0.0117,0.000954,0.193404,0.192450
7,BHZ,G1_W-2,10.0,10.0,3333.000000,4018.849365,83.983009,673.395935,589.413025,0.0152,0.020897,0.167559,0.146662
8,BHZ,G1_W-3,9.0,9.0,3936.000000,3994.022461,17.004000,1095.632202,1078.628296,0.0156,0.004257,0.274318,0.270061
9,BHZ,G1_W-4,9.0,9.0,3152.000000,2867.921143,16.214399,515.941040,499.726654,0.0126,0.005654,0.179901,0.174247
